In [1]:
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Evaluator, Pusher
from tfx.orchestration import pipeline
from tfx.proto import trainer_pb2
import tensorflow_data_validation as tfdv
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
import pprint as pp
import os
from absl import logging

2025-12-11 14:34:39.958176: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/laura/VSCodeProj/ML_project/.venv/lib/python3.9/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/laura/VSCodeProj/ML_project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
PIPELINE_NAME = "HAR-simple"
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
#METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

logging.set_verbosity(logging.INFO)

In [3]:
DATA_ROOT = os.path.join('../data/root/')
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [4]:
example_gen = CsvExampleGen(input_base=DATA_ROOT)
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data ../data/root/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 103
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [7]:
artifact = example_gen.outputs['examples'].get()[0]
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: pipelines/HAR-simple/CsvExampleGen/examples/103


In [8]:
train_uri = os.path.join(artifact.uri, 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [9]:
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    SOURCE: Akila Somasundaram at https://medium.com/data-science/tensorflow-transform-ensuring-seamless-data-preparation-in-production-99ffcf49f535
    '''

    # initialize an empty list
    records = []

    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):

        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()

        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()

        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)

        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))

        # append to the records list
        records.append(example_dict)

    return records

In [10]:
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'activity_id': {'bytesList': {'value': ['c2l0dGluZw==']}},
                           'ax': {'floatList': {'value': [-0.001]}},
                           'ay': {'floatList': {'value': [0.849]}},
                           'az': {'floatList': {'value': [0.48433334]}},
                           'participant_id': {'bytesList': {'value': ['Wk0=']}},
                           'timestamp': {'floatList': {'value': [1760380200.0]}}}}},
 {'features': {'feature': {'activity_id': {'bytesList': {'value': ['c2l0dGluZw==']}},
                           'ax': {'floatList': {'value': [-0.008]}},
                           'ay': {'floatList': {'value': [0.85]}},
                           'az': {'floatList': {'value': [0.483]}},
                           'participant_id': {'bytesList': {'value': ['Wk0=']}},
                           'timestamp': {'floatList': {'value': [1760380200.0]}}}}},
 {'features': {'feature': {'activity_id': {'bytesList': {'value': ['c2l0dGluZw==']

2025-12-11 14:38:54.049781: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to pipelines/HAR-simple/StatisticsGen/statistics/104/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to pipelines/HAR-simple/StatisticsGen/statistics/104/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 104
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [12]:
context.show(statistics_gen.outputs['statistics'])

In [13]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to pipelines/HAR-simple/SchemaGen/schema/105/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'activity_id',STRING,required,,'activity_id'
'ax',FLOAT,required,,-
'ay',FLOAT,required,,-
'az',FLOAT,required,,-
'participant_id',STRING,required,,'participant_id'
'timestamp',FLOAT,required,,-


,Values
Domain,
'activity_id',"'lying', 'running', 'sitting', 'walking'"
'participant_id',"'LL', 'VV', 'YL', 'YT', 'ZM'"


In [14]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to pipelines/HAR-simple/ExampleValidator/anomalies/106/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to pipelines/HAR-simple/ExampleValidator/anomalies/106/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 106
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [15]:
context.show(example_validator.outputs['anomalies'])

Transform

In [16]:
CONSTANTS_MODULE_FILE = 'constants.py'

In [17]:
%%writefile {CONSTANTS_MODULE_FILE}

NUMERICAL_FEATURES = [
    'ax',
    'ay',
    'az', 
    'timestamp'
]

CATEGORICAL_STRING_FEATURES = [
    'participant_id'
]

# Number of vocabulary terms used for encoding categorical features.
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized categorical are hashed.
# no bucket so set to 0
OOV_SIZE = 0

# Keys
LABEL_KEY = 'activity_id'

#rename feature keys to prevent clash with raw keys names
def t_name(key):
  return key + '_xf'

Overwriting constants.py


In [18]:
TRANSFORM_MODULE_FILE = 'transform.py'

In [19]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
# constant are saved in cached so for updated changes, reload imports and file
import constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(constants)

_NUMERICAL_FEATURES = constants.NUMERICAL_FEATURES
_CATEGORICAL_STRING_FEATURES = constants.CATEGORICAL_STRING_FEATURES
_VOCAB_SIZE = constants.VOCAB_SIZE
_OOV_SIZE = constants.OOV_SIZE
_LABEL_KEY = constants.LABEL_KEY

# one-hot tensor to encode categorical features
# returns dense one hot tensor as flost list
# x is dense tensor
# key is string key for feature input 
def make_one_hot(x, key):
  integerized = tft.compute_and_apply_vocabulary(x,
          top_k=_VOCAB_SIZE,
          num_oov_buckets=_OOV_SIZE,
          vocab_filename=key, name=key)
  depth = (
      tft.experimental.get_vocabulary_size_by_name(key) + _OOV_SIZE)
  one_hot_encoded = tf.one_hot(
      integerized,
      depth=tf.cast(depth, tf.int32),
      on_value=1.0,
      off_value=0.0)
  return tf.reshape(one_hot_encoded, [-1, depth])

# fill missing vals of x with ''/0
def fill_in_missing(x):
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


# preprocessing inputs
def preprocessing_fn(inputs):
  outputs = {}
  for key in _NUMERICAL_FEATURES:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[constants.t_name(key)] = tft.scale_to_z_score(
        fill_in_missing(inputs[key]), name=key)

    for key in _CATEGORICAL_STRING_FEATURES:
      outputs[constants.t_name(key)] = make_one_hot(fill_in_missing(inputs[key]), key)

  # Convert label to int
  # 0: lying, 1: running, 2: sitting, 3: walking
  outputs[constants.LABEL_KEY] = tft.compute_and_apply_vocabulary(
      fill_in_missing(inputs[constants.LABEL_KEY]),
      top_k=None,                 # keep all classes
      num_oov_buckets=0,          # no out-of-vocab
      vocab_filename=constants.LABEL_KEY
  )
  # returns feature to transformed feature operations
  return outputs

Overwriting transform.py


In [20]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE))
context.run(transform, enable_cache=True)

INFO:absl:Generating ephemeral wheel package for '/Users/laura/VSCodeProj/ML_project/tfx/transform.py' (including modules: ['constants', 'transform', 'trainer']).
INFO:absl:User module package has hash fingerprint version 20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f.
INFO:absl:Executing: ['/Users/laura/VSCodeProj/ML_project/.venv/bin/python', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpkjjzpq39/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpbmik9k6b', '--dist-dir', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpmmcerevr']
INFO:absl:Successfully built user code wheel distribution at 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl'; target user module is 'transform'.
INFO:absl:Full user module path is 'transform@pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying constants.py -> build/lib
copying transform.py -> build/lib
copying trainer.py -> build/lib
installing to /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpbmik9k6b
running install
running install_lib
copying build/lib/constants.py -> /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpbmik9k6b
copying build/lib/transform.py -> /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpbmik9k6b
copying build/lib/trainer.py -> /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpbmik9k6b
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCE

INFO:absl:Running executor for Transform
INFO:absl:Analyze the 'train' split and transform all splits when splits_config is not set.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'transform@pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl', 'preprocessing_fn': None} 'preprocessing_fn'
INFO:absl:Installing 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/Users/laura/VSCodeProj/ML_project/.venv/bin/python', '-m', 'pip', 'install', '--target', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpr3k7hknc', 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl']


Processing ./pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl


INFO:absl:Successfully installed 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl'.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'transform@pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl', 'stats_options_updater_fn': None} 'stats_options_updater_fn'
INFO:absl:Installing 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/Users/laura/VSCodeProj/ML_project/.venv/bin/python', '-m', 'pip', 'install', '--target', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpligw84w7', 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl']


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 107
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [21]:
transform.outputs

{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_componen

In [23]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transformed_metadata', 'transform_fn', 'metadata']

In [24]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "activity_id"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "ax_xf"
    value {
      float_list {
        value: -0.2709307372570038
      }
    }
  }
  feature {
    key: "ay_xf"
    value {
      float_list {
        value: 1.4972364902496338
      }
    }
  }
  feature {
    key: "az_xf"
    value {
      float_list {
        value: 0.5400288701057434
      }
    }
  }
  feature {
    key: "participant_id_xf"
    value {
      float_list {
        value: 0.0
        value: 0.0
        value: 1.0
        value: 0.0
        value: 0.0
      }
    }
  }
  feature {
    key: "timestamp_xf"
    value {
      float_list {
        value: -1.5227463245391846
      }
    }
  }
}

features {
  feature {
    key: "activity_id"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "ax_xf"
    value {
      float_list {
        value: -0.28597861528396606
      }
    }
  }
  feature {
    k

2025-12-11 14:57:57.362440: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Trainer

In [25]:
TRAINER_MODULE_FILE = 'trainer.py'

In [26]:
%%writefile {TRAINER_MODULE_FILE}

from typing import Dict, List, Text

import os
import glob
from absl import logging

import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_transform import TFTransformOutput
# constants are cached so for updated, need to reload imports and file
import constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(constants)


_LABEL_KEY = constants.LABEL_KEY

_BATCH_SIZE = 40

# features and label for tuning/training
def input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      tf_transform_output.transformed_metadata.schema)

# serving default signature
def get_tf_examples_serving_signature(model, tf_transform_output):

  # to track the layers in the model in order to save it
  model.tft_layer_inference = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_example):
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    # Remove label feature since these will not be present at serving time
    raw_feature_spec.pop(_LABEL_KEY)
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_inference(raw_features)
    logging.info('serve_transformed_features = %s', transformed_features)

    outputs = model(transformed_features)
    # returns outputs to use in serving sign
    return {'outputs': outputs}

  return serve_tf_examples_fn

# transform features, used by tfx validator
def get_transform_features_signature(model, tf_transform_output):

  # To track the layers in the model in order to save it
  model.tft_layer_eval = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def transform_features_func(serialized_tf_example):
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_eval(raw_features)
    logging.info('eval_transformed_features = %s', transformed_features)
    # returns the transformed_features to be fed as input to evaluator
    return transformed_features

  return transform_features_func


# export keras model with 2 signatures, default and transform feature
def export_serving_model(tf_transform_output, model, output_dir):
  # layer saved to the model for keras tracking purpases
  model.tft_layer = tf_transform_output.transform_features_layer()

  signatures = {
      'serving_default':
          get_tf_examples_serving_signature(model, tf_transform_output),
      'transform_features':
          get_transform_features_signature(model, tf_transform_output),
  }

  tf.saved_model.save(model, output_dir, signatures=signatures)


# build DNN keras model for classifying 4 HAR activities
def build_keras_model(tf_transform_output: TFTransformOutput) -> tf.keras.Model:
  feature_spec = tf_transform_output.transformed_feature_spec().copy()
  feature_spec.pop(_LABEL_KEY)

  inputs = {}
  for key, spec in feature_spec.items():
      if isinstance(spec, tf.io.VarLenFeature):
          inputs[key] = tf.keras.layers.Input(
              shape=[None], name=key, dtype=spec.dtype, sparse=True)
      elif isinstance(spec, tf.io.FixedLenFeature):
          inputs[key] = tf.keras.layers.Input(
              shape=spec.shape or [1], name=key, dtype=spec.dtype)
      else:
          raise ValueError('Spec type is not supported: ', key, spec)

  x = tf.keras.layers.Concatenate()(tf.nest.flatten(inputs))
  x = tf.keras.layers.Dense(100, activation='relu')(x)
  x = tf.keras.layers.Dense(70, activation='relu')(x)
  x = tf.keras.layers.Dense(50, activation='relu')(x)
  x = tf.keras.layers.Dense(20, activation='relu')(x)
  output = tf.keras.layers.Dense(4)(x)  # 4 logits for 4 activities
  return tf.keras.Model(inputs=inputs, outputs=output)


# TFX Trainer
def run_fn(fn_args: tfx.components.FnArgs):
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = input_fn(fn_args.train_files, fn_args.data_accessor,
                            tf_transform_output, _BATCH_SIZE)
  eval_dataset = input_fn(fn_args.eval_files, fn_args.data_accessor,
                           tf_transform_output, _BATCH_SIZE)

  model = build_keras_model(tf_transform_output)
    
  # compile model with multiclass metrics from tf keras metrics
  model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name='overall_accuracy'),
        # per-class accuracy
        tf.keras.metrics.SparseCategoricalAccuracy(name='lying_accuracy'),
        tf.keras.metrics.SparseCategoricalAccuracy(name='walking_accuracy'),
        tf.keras.metrics.SparseCategoricalAccuracy(name='sitting_accuracy'),
        tf.keras.metrics.SparseCategoricalAccuracy(name='running_accuracy'),])

  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  # Export the model.
  export_serving_model(tf_transform_output, model, fn_args.serving_model_dir)

Overwriting trainer.py


In [27]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000))
context.run(trainer, enable_cache=True)


INFO:absl:Generating ephemeral wheel package for '/Users/laura/VSCodeProj/ML_project/tfx/trainer.py' (including modules: ['constants', 'transform', 'trainer']).
INFO:absl:User module package has hash fingerprint version 20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f.
INFO:absl:Executing: ['/Users/laura/VSCodeProj/ML_project/.venv/bin/python', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpy7ex2o69/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmphp2ejtnn', '--dist-dir', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpai54nu88']
INFO:absl:Successfully built user code wheel distribution at 'pipelines/HAR-simple/_wheels/tfx_user_code_Trainer-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl'; target user module is 'trainer'.
INFO:absl:Full user module path is 'trainer@pipelines/HAR-simple/_wheels/tfx_user_code_Trainer-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d73991

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying constants.py -> build/lib
copying transform.py -> build/lib
copying trainer.py -> build/lib
installing to /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmphp2ejtnn
running install
running install_lib
copying build/lib/constants.py -> /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmphp2ejtnn
copying build/lib/transform.py -> /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmphp2ejtnn
copying build/lib/trainer.py -> /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmphp2ejtnn
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writi

INFO:absl:udf_utils.get_fn {'train_args': '{\n  "num_steps": 10000\n}', 'eval_args': '{\n  "num_steps": 5000\n}', 'module_file': None, 'run_fn': None, 'custom_config': 'null', 'module_path': 'trainer@pipelines/HAR-simple/_wheels/tfx_user_code_Trainer-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl'} 'run_fn'
INFO:absl:Installing 'pipelines/HAR-simple/_wheels/tfx_user_code_Trainer-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/Users/laura/VSCodeProj/ML_project/.venv/bin/python', '-m', 'pip', 'install', '--target', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpjqwasrc6', 'pipelines/HAR-simple/_wheels/tfx_user_code_Trainer-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl']


Processing ./pipelines/HAR-simple/_wheels/tfx_user_code_Trainer-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl


INFO:absl:Successfully installed 'pipelines/HAR-simple/_wheels/tfx_user_code_Trainer-0.0+20ebd14c5aa4b4229fcf1a4f5170ed2b9d96b6d7399129fa0795fe4bccc57b1f-py3-none-any.whl'.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Feature activity_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature ax has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature ay has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature az has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature participant_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature timestamp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:serve_transformed_features = {'participant_id_xf': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:4' shape=(None, 5) dtype=float32>, 'ay_xf': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:2' shape=(None, 1) dtype=float32>, 'timestamp_xf': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:5' shape=(None, 1) dtype=float32>, 'az_xf': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:3' shape=(None, 

INFO:tensorflow:Assets written to: pipelines/HAR-simple/Trainer/model/108/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/HAR-simple/Trainer/model/108/Format-Serving/assets
INFO:absl:Writing fingerprint to pipelines/HAR-simple/Trainer/model/108/Format-Serving/fingerprint.pb
INFO:absl:Training complete. Model written to pipelines/HAR-simple/Trainer/model/108/Format-Serving. ModelRun written to pipelines/HAR-simple/Trainer/model_run/108
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 108
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [28]:
model_artifact = trainer.outputs['model'].get()[0]
print("SavedModel exists at root:", os.path.exists(model_artifact.uri))
print("Contents:", os.listdir(model_artifact.uri))  # should include saved_model.pb, variables/

format_serving_dir = os.path.join(model_artifact.uri, "Format-Serving")
print("Format-Serving exists:", os.path.exists(format_serving_dir))
print("Contents of Format-Serving:", os.listdir(format_serving_dir))

SavedModel exists at root: True
Contents: ['Format-Serving']
Format-Serving exists: True
Contents of Format-Serving: ['fingerprint.pb', 'variables', 'saved_model.pb', 'assets']


In [29]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

#we only need to use load_ext the first time at the notebook session
# %load_ext tensorboard
#use reload afterwards
%reload_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}